<a href="https://colab.research.google.com/github/quentin12349876/hello-world/blob/main/solver/CNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
path = "/content/initial_conditions (5).csv"
df = pd.read_csv(path, encoding='latin1')
array_data = df.to_numpy()  # converting data to numpy array
df.columns = df.columns.str.strip().str.replace('\ufeff', '')
df = df.rename(columns={
    df.columns[0]: 'concentration',
    df.columns[1]: 'distance'})
df['concentration'] = pd.to_numeric(df['concentration'], errors='coerce')
df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
df = df.dropna()
df = df.sort_values(by='concentration')
df = df.drop_duplicates(subset='concentration')
x = df['concentration'].to_numpy()
y = df['distance'].to_numpy()
f = interp1d(x, y, kind='linear', bounds_error=False)
num_between = 5
x_new = np.linspace(x.min(), x.max(), (len(x) - 1) * num_between + 1)
y_new = f(x_new)

extended_df = pd.DataFrame({
    'concentration': x_new,
    'distance': y_new})
extended_df.to_csv("extended_data.csv", index=False)
print(x_new,y_new)

class AdvectionSolver:
  #Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0


    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_grid = np.arange(0, self.L + self.dx, self.dx)
        self.num_points = len(self.x_grid)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = theta_initial.copy()
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):

        return self._solve_implicit(theta_initial, boundary_theta)

solver_A = AdvectionSolver(L, dx, dt, T, U)
res_A = solver_A.compute_solution(theta_initial_A, boundary_theta=100.0)

fig, ax = plt.subplots(figsize=(4,3))

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff() # interactive off

def animate(t):
  plt.cla()
  y = res_A[t]
  plt.plot(x_grid_A, y)
  plt.xlim(0,L)
  plt.ylim(0, res_A[t].max())
  ax.set_title("Concentration of pollutant at t = " + str(dt*t) + "s", fontsize=10, verticalalignment='top')
  plt.xlabel("x", fontsize=8)
  plt.ylabel("Concentration", fontsize=8)

matplotlib.animation.FuncAnimation(fig, animate, frames=len(res_A))

[ 0.   0.2  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6
  2.8  3.   3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8  5.   5.2  5.4
  5.6  5.8  6.   6.2  6.4  6.6  6.8  7.   7.2  7.4  7.6  7.8  8.   8.2
  8.4  8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.
 11.2 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8
 14.  14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8 16.  16.2 16.4 16.6
 16.8 17.  17.2 17.4 17.6 17.8 18.  18.2 18.4 18.6 18.8 19.  19.2 19.4
 19.6 19.8 20. ] [300.  242.  184.  126.   68.   10.   10.   10.   10.   10.   10.   10.
  10.   10.   10.   10.   10.   10.   10.   10.   10.    9.6   9.2   8.8
   8.4   8.    8.    8.    8.    8.    8.    8.    8.    8.    8.    8.
   8.    8.    8.    8.    8.    7.8   7.6   7.4   7.2   7.    7.    7.
   7.    7.    7.    7.    7.    7.    7.    7.   12.6  18.2  23.8  29.4
  35.   39.4  43.8  48.2  52.6  57.   61.6  66.2  70.8  75.4  80.   81.
  82.   83.   84.   85.   84.   83.   82.   81.   80.

NameError: name 'L' is not defined